# Поиск и загрузка твиттов по ключевому слову
https://dev.twitter.com/rest/reference/get/search/tweets

In [48]:
from IPython.core.display import clear_output
from urllib import unquote
import re

In [2]:
import twitter

In [3]:
# Go to http://dev.twitter.com/apps/new to create an app and get values
# for these credentials, which you'll need to provide in place of these
# empty string values that are defined as placeholders.
# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation.

CONSUMER_KEY = ''
CONSUMER_SECRET = ''
OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''

In [26]:
auth = twitter.oauth.OAuth(OAUTH_TOKEN,
                           OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY,
                           CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth, retry=True)

Задаем необходимые условия поиска

In [51]:
# По какому ключевому слову ищем
q = 'Russia'

# Сколько твиттов скачивать
count = 300

# Сюда сохраняем твитты
all_tweets = []

Посылаем запросы на сервер твиттера пока не будет скачано заданое кол-во твиттов в переменной **count**

In [52]:
%%time
L = 0
while L<count:
    text_list = []
    
    # 1. Запрашиваем 100 твиттов
    try:
        search_results = twitter_api.search.tweets(q=q, count=100)
    except twitter.TwitterHTTPError as e:
            print e
            break
    
    statuses = search_results['statuses']
    
    # 1.1. Продолжаем запросы, если есть next_results
    for i in range(10):
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError, e: # No more results when next_results doesn't exist
            break

        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([kv.split('=') for kv in unquote(next_results[1:]).split("&") ])    

        try:
            search_results = twitter_api.search.tweets(**kwargs)
        except twitter.TwitterHTTPError as e:
            print e
            break
        
        statuses += search_results['statuses']

    # 2. Создаем список твиттов 
    for i in range(len(statuses)):
        text_list.append(statuses[i]['text'])

    # 3. Выбираем только уникальные твитты
    text_list = text_list + all_tweets
    text_set = set(text_list)
    all_tweets = list(text_set)
    L = len(all_tweets)
    
    clear_output(wait=True)
    print 'Tweets downloaded:', L
    


Tweets downloaded: 322
Wall time: 1min 7s


## Сохраняем скачанные твитты в файл для дальнейшего анализа

In [34]:
import pickle

In [35]:
pickle.dump(all_tweets, open('all_tweets.pkl', 'wb'), protocol=2)

Загрузка сохраненных данных

In [36]:
my_list2 = pickle.load(open('all_tweets.pkl', 'rb'))
len(my_list2)

10013